# Sparkgeo GoGeomatics Workshop

Welcome to the workshop on metadata creation for PDAL datasets using Python! We'll explore how to generate a SpatioTemporal Asset Catalog (STAC) to describe point cloud data effectively.

Talk about STAC and stactools-packages

Before we begin, we need to ensure that our environment is set up with the necessary libraries. We'll be using pdal, pystac, pyproj, shapely, and stactools.

In [2]:
# !pip install pdal pystac pyproj shapely stactools requests

In [49]:
import datetime
import json
import logging
import os.path
import re
import requests
from dateutil.parser import parse
from typing import Any, Dict, List, Optional
from tempfile import TemporaryDirectory

import pdal
import pystac
from pyproj import CRS
from pystac.extensions.pointcloud import (
    PointcloudExtension,
    Schema,
    SchemaType,
    Statistic,
)
from pystac.extensions.projection import ProjectionExtension
from pystac.link import Link
from pystac.provider import Provider, ProviderRole
from shapely.geometry import box, mapping, shape
from stactools.core.projection import reproject_geom

## Create a STAC Collection
With our imports in place, let's create a STAC Collection. A Collection is a group of STAC Items that share the same metadata and providers. We will define some constants for the data we are using and create a function that fetches metadata and constructs the Collection.

In [51]:
# Constants
OPEN_CANADA_ID = "7069387e-9986-4297-9f55-0288e9676947"

STAC_ID = "nrcan-canelevation"

METADATA_URL = (
    f"https://open.canada.ca/data/api/action/package_show?id={OPEN_CANADA_ID}"
)
PROVIDER_URL = f"https://open.canada.ca/data/en/dataset/{OPEN_CANADA_ID}"
KEYWORDS = [
    "Point Cloud",
    "North America",
    "Canada",
    "Remote Sensing",
    "LiDAR",
    "CanElevation",
    "NRCAN",
    "Open Canada",
]

# Set temporary directory to store source data
tmp_dir = TemporaryDirectory()
img_path = os.path.join(tmp_dir.name, 'image.tif')

In [27]:
# Check what is available in the metadata
metadata_response = requests.get(METADATA_URL)
remote_metadata = metadata_response.json()["result"]

In [30]:
def create_collection(metadata_url: str = METADATA_URL) -> pystac.Collection:
    """Create a STAC Collection for CanElevation

    Args:
        metadata_url (str): The url from which to get metadata

    Returns:
        pystac.Collection: pystac collection object
    """

    provider = Provider(
        name=remote_metadata["organization"]["title"],
        roles=[
            ProviderRole.HOST,
            ProviderRole.LICENSOR,
            ProviderRole.PROCESSOR,
            ProviderRole.PRODUCER,
        ],
        url=PROVIDER_URL,
    )

    # Get the geometry and make a bbox
    geo = shape(json.loads(remote_metadata["spatial"]))
    bbox = list(geo.bounds)

    # Function to clean up datetimes...
    def get_datetime(dt_text):
        dt = parse(dt_text)
        assert isinstance(dt, datetime.datetime)
        return dt.astimezone(datetime.timezone.utc)

    datetime_start = get_datetime(
        remote_metadata["time_period_coverage_start"]
    )

    if "time_period_coverage_end" in remote_metadata:
        datetime_end = get_datetime(
            remote_metadata["time_period_coverage_end"]
        )
    else:
        datetime_end = None  # Ongoing data capture

    # Create an extent object
    extent = pystac.Extent(
        pystac.SpatialExtent([bbox]),
        pystac.TemporalExtent([[datetime_start, datetime_end]]),
    )

    collection = pystac.Collection(
        id=STAC_ID,
        title=remote_metadata["title"],
        description=remote_metadata["notes"],
        providers=[provider],
        license=remote_metadata["license_id"],
        extent=extent,
        catalog_type=pystac.CatalogType.RELATIVE_PUBLISHED,
        keywords=KEYWORDS,
    )

    collection.add_link(
        Link(rel="license", target=remote_metadata["license_url"], title=remote_metadata["license_title"])
    )

    return collection

In [35]:
collection = create_collection()
collection

<Collection id=nrcan-canelevation>

### Create a STAC Item
A STAC Item is a construct that holds specific metadata about a single spatial asset. Here, we'll write a function to create a STAC Item for our point cloud data.



In [40]:
def create_item(
    href: str,
    additional_providers: Optional[List[Provider]] = None,
    quick: bool = True,
) -> pystac.Item:
    """Creates a STAC Item from a point cloud.

    Args:
        href (str): The href to the point cloud.
        quick (bool): Do a quick look into the point cloud

    Return:
        pystac.Item: A STAC Item representing this point cloud.
    """
    reader = href

    pipeline = pdal.Pipeline(json.dumps([reader, {"type": "filters.head", "count": 0}]))

    if quick:
        metadata = pipeline.quickinfo
    else:
        pipeline.execute()
        metadata = pipeline.metadata["metadata"]

    # find the pdal reader user
    def _extract_reader_key(metadata: Dict[str, Any]) -> str:
        for key in metadata.keys():
            if key.startswith("readers"):
                return key
        raise Exception("Could not find reader key in pipeline metadata")

    reader_key = _extract_reader_key(metadata)
    metadata = metadata[reader_key]
    id = os.path.splitext(os.path.basename(href))[0]
    encoding = os.path.splitext(href)[1][1:]

    if quick:
        spatialreference = CRS.from_wkt(metadata["srs"]["compoundwkt"])
    else:
        spatialreference = CRS.from_string(metadata["spatialreference"])

    if quick:
        original_bbox = box(
            metadata["bounds"]["minx"],
            metadata["bounds"]["miny"],
            metadata["bounds"]["maxx"],
            metadata["bounds"]["maxy"],
        )
    else:
        original_bbox = box(
            metadata["minx"], metadata["miny"], metadata["maxx"], metadata["maxy"]
        )

    geometry = reproject_geom(
        spatialreference, "EPSG:4326", mapping(original_bbox), precision=6
    )
    bbox = list(shape(geometry).bounds)

    # THESE DATES ARE DIFFERENT!!!
    if quick:
        # add try statement here because there is an error
        try:
            filedate = re.findall(r"\d{8}", id)[0]
        except IndexError:
            print(id)
            filedate = "19010101"
        dt = datetime.datetime.strptime(filedate, "%Y%m%d")

    else:
        dt = datetime.datetime(metadata["creation_year"], 1, 1) + datetime.timedelta(
            metadata["creation_doy"] - 1
        )
    id_name = os.path.splitext(id)[0]
    item = pystac.Item(
        id=id_name, geometry=geometry, bbox=bbox, datetime=dt, properties={}
    )

    # Create a campaign property
    pattern = r"([A-Z]{2}_\w+_\d{4})"
    match = re.search(pattern, os.path.splitext(id_name)[0])
    if match is not None:
        item.properties["canelevation:campaign"] = match.group(1)
    else:
        item.properties["canelevation:campaign"] = "XXX"

    item.add_asset(
        "pointcloud",
        pystac.Asset(
            href=href,
            media_type="application/vnd.laszip+copc",
            roles=["data"],
            title=f"{encoding} point cloud",
        ),
    )

    # if additional_providers:
    #     item.common_metadata.providers.extend(additional_providers)

    pc_ext = PointcloudExtension.ext(item, add_if_missing=True)
    pc_ext.count = metadata["num_points"] if quick else metadata["count"]
    pc_ext.type = "LiDAR"
    pc_ext.encoding = encoding

    if quick:
        # We are filling the data with empty values. Since we did a quicklook, we only looked
        # at the header and not the full dataset.
        pc_ext.schemas = [
            Schema.create(dim, 0, SchemaType.SIGNED)
            for dim in metadata["dimensions"].split(",")
        ]
    else:
        schema = pipeline.schema["schema"]["dimensions"]
        pc_ext.schemas = [Schema(schema) for schema in schema]

    epsg = spatialreference.to_epsg()
    if epsg:
        proj_ext = ProjectionExtension.ext(item, add_if_missing=True)
        proj_ext.epsg = epsg
        proj_ext.wkt2 = spatialreference.to_wkt()
        proj_ext.bbox = list(original_bbox.bounds)

    return item

In [50]:
item = create_item('https://ftp-maps-canada-ca.s3.amazonaws.com/pub/elevation/pointclouds_nuagespoints/NRCAN/Fort_McMurray_2018/AB_FortMcMurray2018_20180518_NAD83CSRS_UTMZ12_1km_E4760_N62940_CQL1_CLASS.copc.laz')
item

<Item id=AB_FortMcMurray2018_20180518_NAD83CSRS_UTMZ12_1km_E4760_N62940_CQL1_CLASS>

In [43]:
collection.add_item(item)
collection.describe()

<Link rel=item target=<Item id=AB_FortMcMurray2018_20180518_NAD83CSRS_UTMZ12_1km_E4760_N62940_CQL1_CLASS>>

In [46]:
# set the href and save the file
print(collection.get_self_href() is None)
print(item.get_self_href() is None)

True
True


In [55]:
collection .normalize_hrefs(os.path.join(tmp_dir.name, "stac"))
collection.save(catalog_type=pystac.CatalogType.SELF_CONTAINED)
!ls {tmp_dir.name}/stac/*



/var/folders/fj/p1pfrpwj2bs94wn5lpxbm8g80000gn/T/tmpm27uj8un/stac/collection.json

/var/folders/fj/p1pfrpwj2bs94wn5lpxbm8g80000gn/T/tmpm27uj8un/stac/AB_FortMcMurray2018_20180518_NAD83CSRS_UTMZ12_1km_E4760_N62940_CQL1_CLASS:
AB_FortMcMurray2018_20180518_NAD83CSRS_UTMZ12_1km_E4760_N62940_CQL1_CLASS.json


In [54]:
# check that everything is there
with open(collection.self_href) as f:
    print(f.read())

{
  "type": "Collection",
  "id": "nrcan-canelevation",
  "stac_version": "1.0.0",
  "description": "The LiDAR Point Clouds is a product that is part of the CanElevation Series created to support the National Elevation Data Strategy implemented by NRCan.  \n  \n  This product contains point clouds from various airborne LiDAR acquisition projects conducted in Canada. These airborne LiDAR acquisition projects may have been conducted by NRCan or by various partners. The LiDAR point cloud data is licensed under an open government license and has been incorporated into the National Elevation Data Strategy.  \n  \n  Point cloud files are distributed by LiDAR acquisition project without integration between projects.  \n  \n  The point cloud files are distributed using the compressed .LAZ / Cloud Optimized Point Cloud (COPC) format. The COPC open format is an octree reorganization of the data inside a .LAZ 1.4 file. It allows efficient use and visualization rendering via HTTP calls (e.g. via t